In [1]:
import pandas as pd
import numpy as np
import pickle
import vk_api
vk_session = vk_api.VkApi(token="8e57ea0d696bb77a99aa87c3076a19941a630bece5fb14d50073fa0d1c9d0d0b9abe7e0474628702089f2")

In [2]:
vk_session.authorization()

In [3]:
cached_ids = {}#pickle.load(open("cache2.p", "rb"))

In [4]:
indexes = np.load("indexex.npy")

In [5]:
indexes

array([1899, 2523,  318, ..., 2326, 1971, 3170])

In [6]:
data = pd.read_csv("communities.csv").to_dict()

In [7]:
def get_many_of_public(group_link, vk_session):
    global cached_ids
    
    shortname = group_link[group_link.rfind("/")+1:]
    if "public" in shortname:
        shortname = shortname[6:]
    print(group_link)

    r = vk_session.method("groups.getById", {"group_ids": shortname})
    group_id = r[0]["id"]
    
    if group_id not in cached_ids.keys():
    
        """Все сообщества подписчиков сообщества"""
        users = vk_api.VkTools(vk_session).get_all("groups.getMembers", 100, {'group_id':group_id})['items']
        k = int(len(users) / 1000)
        if k:
            users = users[::k]

        data_for_clustering = []
        users_publicpages = []
        with vk_api.VkRequestsPool(vk_session) as pool:
            
            for i in range(0, len(users), 1000):
                batch = users[i:i+1000]
                data_for_clustering.append(pool.method('users.get', {
                        "user_ids": batch,
                        "fields": "sex,age,education,universities,schools,interests,music,movies,bdate,city,country"
                    }))

                for user_id in batch:
                    users_publicpages.append(pool.method('users.getSubscriptions', {
                                "user_id": user_id
                    }))

        data_for_clustering = [x for x in [x.result for x in data_for_clustering]]
        t = []
        for user in users_publicpages:
            try:
                t.append(user.result['groups']['items'])
            except:
                pass
        users_publicpages = t
        
        wall50 = vk_session.method("wall.get", {"owner_id": group_id, "filter": "owner", "count": 100})['items'][::2]
        
        result = data_for_clustering, users_publicpages, wall50
        cached_ids[group_id] = result
        #pickle.dump(cached_ids, open("cache2.p", "wb"))
        return 
    else:
        return cached_ids[group_id]

In [8]:
from tqdm import tqdm_notebook


for i in tqdm_notebook(indexes[1000:1500]):
    try:
        get_many_of_public(data["link"][i], vk_session)
    except KeyError:
        pass
    except Exception as e:
        print(e)

https://vk.com/ladies.trickss


KeyboardInterrupt: 

In [66]:
1

1